<a href="https://colab.research.google.com/github/kty0307/Blog/blob/main/SP_LOAN_INTRATE_DOWNLOAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **연월입력**

In [8]:
year = 2025     ## ex) 2024
month = 2       ## ex) 1

# 0. 환경설정

In [10]:
import requests
import json
import pandas as pd
import urllib.parse
import urllib3
import ssl
from urllib.parse import unquote
from requests.adapters import HTTPAdapter
from urllib3.util.ssl_ import create_urllib3_context
from bs4 import BeautifulSoup
from google.colab import files

In [11]:
class TLSAdapter(requests.adapters.HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context()
        ctx.set_ciphers("AES128-SHA256")
        kwargs["ssl_context"] = ctx
        return super(TLSAdapter, self).init_poolmanager(*args, **kwargs)

In [12]:
class NewTLSAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context()
        ctx.set_ciphers("AES128-SHA256")  # 원하는 Cipher Suite 설정
        kwargs["ssl_context"] = ctx
        return super().init_poolmanager(*args, **kwargs)

In [13]:
def subtract_month(df, col_name, new_col_name):
    df[new_col_name] = pd.to_datetime(df[col_name], format='%Y%m') - pd.DateOffset(months=1)
    df[new_col_name] = df[new_col_name].dt.strftime('%Y%m').astype(int)
    return df

# 1. 저축은행

## 1.1. 신용점수별 금리

In [ ]:
def savings_cs(year, month):
  url = "https://www.fsb.or.kr/ratloanconf_0200.jct"
  payload = {
      "_JSON_": json.dumps({
          "SORT_COLUMN": "",
          "SORT": "",
          "PRE_MONTH_MONEY": "",
          "SUBMIT_MONTH": f"{year:04d}{month:02d}"
      })
  }

  response = requests.post(url, data=payload)

  data = response.text

  json_data = json.loads(data)

  # 데이터 추출 및 DataFrame 생성
  df = pd.DataFrame(json_data['REC'])
  df = df[['BANK_NAME', 'SUBMIT_MONTH', 'A_RATE1_3', 'A_RATE1', 'A_RATE2',
            'A_RATE3', 'A_RATE_AVE']]

  df.columns = ['사명','공시연월','900점대','800점대','700점대','600점대','평균금리']

  df['업권'] = '저축은행업권'

  df = subtract_month(df, '공시연월', '연월')

  df = df[['사명','연월','업권','평균금리','900점대','800점대','700점대','600점대','공시연월']]

  return df

## 1.2. 금리대별 취급비중

In [ ]:
def savings_int(year, month):
    url = "https://www.fsb.or.kr/ratloanconf_0300.jct"

    payload = {
        "SORT": "",
        "SUBMIT_MONTH": f"{year}{month:02}"
    }

    response = requests.post(url, data={"_JSON_": json.dumps(payload)})
    response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    df = pd.DataFrame(data["REC"])  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    desired_columns = ["BANK_NAME", "SUBMIT_MONTH", "HANDING_WEIGHT_10", "HANDING_WEIGHT_12", "HANDING_WEIGHT_14"
                        ,"HANDING_WEIGHT_16", "HANDING_WEIGHT_18", "HANDING_WEIGHT_20"]
    df = df[desired_columns] #원하는 열만 남기기

    df.columns = ['사명','제출연월','10%이하','12%이하','14%이하','16%이하','18%이하','20%이하']

    df['업권'] = '저축은행업권'

    return df

# 2. 신용카드

## 2.1. 카드론_신용점수별 금리

In [ ]:
def card_loan_cs(year, month):
    url = "https://gongsi.crefia.or.kr/portal/creditcard/creditcardDisclosureDetail25Ajax"

    # cgc_seq(월)을 찾기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": 1458,
        "cgcMode": 25,
        "cgcYyyy": year,
        "mcSeq": []
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    pick_month = json.loads(response.text)  # 응답 문자열 반환

    for item in pick_month['configListMm']:
        if item['cgcquarter'] == month:
            cgc_seq = item['cgcSeq']
            break  # 첫 번째로 찾은 값만 출력하고 반복문 종료

    # 실제 데이터를 추출하기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": cgc_seq,
        "cgcMode": 25,
        "cgcYyyy": year,
        "mcSeq": [31, 96, 1, 106, 14, 13, 12, 98, 502, 108, 619, 11, 97, 105, 103, 22]
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    pick_data = data["resultList"]
    df = pd.DataFrame(pick_data)  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    df = df[["mcCompany", "cgcSeq", "cgCardPoint1", "cgCardPoint2", "cgCardPoint3"
                ,"cgCardPoint4", "cgCardPointAvg"]] #원하는 열만 남기기

    df["cgcSeq"] = f"{year}{month:02}" if month < 10 else f"{year}{month}"

    df.columns = ['사명','공시연월','900점대','800점대','700점대','600점대','평균금리']

    df['업권'] = '신용카드업권(카드론)'

    df = subtract_month(df, '공시연월', '연월')

    df = df[['사명','연월','업권','평균금리','900점대','800점대','700점대','600점대','공시연월']]

    return df

## 2.2. 현금서비스_신용점수별 금리

In [ ]:
def card_cash_cs(year, month):
    url = "https://gongsi.crefia.or.kr/portal/creditcard/creditcardDisclosureDetail20Ajax"

    # cgc_seq(월)을 찾기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": 1460,
        "cgcMode": 20,
        "cgcYyyy": year,
        "mcSeq": []
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    pick_month = json.loads(response.text)  # 응답 문자열 반환

    for item in pick_month['configListMm']:
        if item['cgcquarter'] == month:
            cgc_seq = item['cgcSeq']
            break  # 첫 번째로 찾은 값만 출력하고 반복문 종료

    # 실제 데이터를 추출하기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": cgc_seq,
        "cgcMode": 20,
        "cgcYyyy": year,
        "mcSeq": [31, 96, 1, 106, 14, 13, 12, 98, 502, 108, 619, 11, 97, 105, 103, 22]
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    pick_data = data["resultList"]
    df = pd.DataFrame(pick_data)  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    df = df[["mcCompany", "cgcSeq", "cgMoneyPoint1", "cgMoneyPoint2", "cgMoneyPoint3"
                ,"cgMoneyPoint4", "cgMoneyPointAvg"]] #원하는 열만 남기기

    df["cgcSeq"] = f"{year}{month:02}" if month < 10 else f"{year}{month}"

    df.columns = ['사명','공시연월','900점대','800점대','700점대','600점대','평균금리']

    df['업권'] = '신용카드업권(현금서비스)'

    df = subtract_month(df, '공시연월', '연월')

    df = df[['사명','연월','업권','평균금리','900점대','800점대','700점대','600점대','공시연월']]

    return df

# 3. 캐피탈

## 3.1. 신용점수별 금리

In [ ]:
def capital_cs(year, month):
    url = "https://gongsi.crefia.or.kr/portal/creditloan/creditloanDisclosureDetail11/ajax"

    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
          "clgcMode": 11,
          "cardItem": "134,39,40,623,130,41,25,156,6,55,32,58,52,61,57,64",
          "clgcSeq": 521,
          "clgcYyyy": 2024
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    pick_month = json.loads(response.text)  # 응답 문자열 반환

    for item in pick_month['configListMm']:
        if item['clgcquarter'] == month:
            clgc_seq = item['clgcSeq']
            break  # 첫 번째로 찾은 값만 출력하고 반복문 종료

    # 실제 데이터를 추출하기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
          "clgcMode": 11,
          "cardItem": "134,39,40,623,130,41,25,156,6,55,32,58,52,61,57,64",
          "clgcSeq": clgc_seq,
          "clgcYyyy": year
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    pick_data = data["resultList"]
    df = pd.DataFrame(pick_data)  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    df = df[["mcCompany", "clgcSeq", "clgPoint1", "clgPoint2", "clgPoint3"
                ,"clgPoint4", "clgPointAvg"]] #원하는 열만 남기기

    df["clgcSeq"] = f"{year}{month:02}" if month < 10 else f"{year}{month}"

    df.columns = ['사명','공시연월','900점대','800점대','700점대','600점대','평균금리']

    df['업권'] = '캐피탈업권'

    df = subtract_month(df, '공시연월', '연월')

    df = df[['사명','연월','업권','평균금리','900점대','800점대','700점대','600점대','공시연월']]

    return df


# 4. 은행(추후지원예정)

In [18]:
url = "https://finlife.fss.or.kr/finlife/ldng/indvlCrdt/list.do"

headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "cache-control": "max-age=0",
    "connection": "keep-alive",
    "content-type": "application/x-www-form-urlencoded",
    "cookie": "PCID=17264824031226575657613; RC_RESOLUTION=1710*1112; RC_COLOR=30; WMONID=zABVjou7o3F; JSESSIONID_finlife=p8aPuFguAymOD4Dzdh0mWo35ut91fwG0hBBSBT3xEz3oWejmvDKOUcaBUEFtoIPD.amV1c19kb21haW4vaGZzc3dhczFfZmlubGlmZV9oZnNzd2FzY29uMV9SNEg=",
    "host": "finlife.fss.or.kr",
    "origin": "https://finlife.fss.or.kr",
    "referer": "https://finlife.fss.or.kr/finlife/ldng/indvlCrdt/list.do",
    "sec-ch-ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "same-origin",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
}

payload = {
    "pageType": "ajax",
    "menuNo": "700009",
    "pageIndex": "1",
    "pageSize": "50",
    "pageUnit": "50",
    "total": "",
    "topFinGrpNo": "",
    "crdtLendRateType": "",
    "topFinGrpNoNM": "전체",
    "crdtLendRateTypeNM": "",
    "joinWay": "1,2,3,4,5,9",
    "searchKeyword": "",
    "searchCondition": "ALL",
    "crdtPrdtType": "1",
    "crdtPrdtTypeNM": "전체",
    "listOrder": "",
    "logAdd": "Y",
    "menuId": "2000104",
    "BLTN_ID": "BB000000000000000151",
    "searchFocus": "1",
}

try:
    response = requests.post(url, headers=headers, data=payload)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("div", {"id": "ajaxResult"}).find("table")

    if table:
        bank_cs = pd.read_html(str(table))[0]

    else:
        print("테이블을 찾을 수 없습니다.")

except requests.exceptions.RequestException as e:
    print(f"오류 발생: {e}")
except Exception as e:
    print(f"알 수 없는 오류 발생: {e}")

bank_cs = bank_cs.dropna(how='all')

bank_cs['업권'] = '은행업권'
bank_cs['공시연월'] = f"{year}{month:02}" if month < 10 else f"{year}{month}"
bank_cs = subtract_month(bank_cs, '공시연월', '연월')
bank_cs = bank_cs[['금융 회사', '연월', '업권', '900점 초과', '801~ 900점', '701~ 800점', '601~ 700점', '평균금리', '공시연월']]
bank_cs.columns = ['사명', '연월', '업권', '평균금리', '900점대', '800점대', '700점대', '600점대', '공시연월']
bank_cs['평균금리'] = bank_cs['평균금리'].str.replace('%', '')
bank_cs['900점대'] = bank_cs['900점대'].str.replace('%', '')
bank_cs['800점대'] = bank_cs['800점대'].str.replace('%', '')
bank_cs['700점대'] = bank_cs['700점대'].str.replace('%', '')
bank_cs['600점대'] = bank_cs['600점대'].str.replace('%', '')


<ipython-input-18-c17b6b670408>:56: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  bank_cs = pd.read_html(str(table))[0]


# 5. 데이터 처리 및 다운로드

In [19]:
bank_cs

,사명,연월,업권,평균금리,900점대,800점대,700점대,600점대,공시연월
0,국민은행,202501,은행업권,4.51,5.07,5.56,6.23,4.68,202502
2,중소기업은행,202501,은행업권,4.65,5.00,5.72,6.32,4.80,202502
4,주식회사 카카오뱅크,202501,은행업권,4.77,4.91,5.59,6.12,4.88,202502
6,주식회사 케이뱅크,202501,은행업권,4.60,5.61,6.14,-,4.94,202502
8,하나은행,202501,은행업권,4.88,5.73,6.77,7.55,5.16,202502
10,신한은행,202501,은행업권,4.85,6.04,7.56,9.45,5.27,202502
12,농협은행주식회사,202501,은행업권,5.16,6.00,6.89,8.17,5.48,202502
14,토스뱅크 주식회사,202501,은행업권,5.07,6.35,7.51,9.27,5.63,202502
16,수협은행,202501,은행업권,5.60,5.97,-,-,5.67,202502
18,한국스탠다드차타드은행,202501,은행업권,5.23,7.11,8.18,8.14,5.82,202502


In [ ]:
file_names = f'SP_INTRATE_{year}{month:02d}.csv'

In [ ]:
df_1 = savings_cs(year, month)
df_2 = card_loan_cs(year, month)
df_3 = card_cash_cs(year, month)
df_4 = capital_cs(year, month)
df_5 = bank_cs

df = pd.concat([df_1, df_2, df_3, df_4, df_5], ignore_index=True)
df.to_csv(file_names, index=False)

In [ ]:
files.download(file_names)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>